In [ ]:
import os
import asyncio
from nano_graphrag import GraphRAG
import networkx as nx

# ✅ STEP 1 – Carica il testo da un file
with open("data/alleanze.txt", "r", encoding="utf-8") as f:
    text = f.read()

# ✅ STEP 2 – Imposta la tua chiave API OpenAI
os.environ["OPENAI_API_KEY"] = "your_api_key"  # Sostituisci con la tua chiave reale

# ✅ STEP 3 – Inizializza GraphRAG
graphrag = GraphRAG(
    working_dir="./nano_cache",
    enable_local=True,
    tokenizer_type="tiktoken",
    tiktoken_model_name="gpt-3.5",  # usa gpt-4o solo se abilitato nel tuo account
    graph_cluster_algorithm=None  # evitiamo errore sul clustering
)

# ✅ STEP 4 – Inserisci il documento (solo in ambienti Jupyter: usa `await`)
await graphrag.ainsert(text)

# ✅ STEP 5 – Carica il grafo salvato da file
G = nx.read_graphml("./nano_cache/graph_chunk_entity_relation.graphml")

# ✅ STEP 6 – Stampa le triplette soggetto --[relazione]--> oggetto
print("\n📌 Triplette estratte:")
for u, v, data in G.edges(data=True):
    relation = data.get("description", "(nessuna relazione)")
    print(f"{u} --[{relation}]--> {v}")



INFO:nano-graphrag:Loading tokenizer: type='tiktoken', name='gpt-3.5'
INFO:nano-graphrag:Load KV full_docs with 1 data
INFO:nano-graphrag:Load KV text_chunks with 1 data
INFO:nano-graphrag:Load KV llm_response_cache with 4 data
INFO:nano-graphrag:Load KV community_reports with 2 data
INFO:nano-graphrag:Loaded graph from ./nano_cache\graph_chunk_entity_relation.graphml with 8 nodes, 12 edges
INFO:nano-vectordb:Load (8, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './nano_cache\\vdb_entities.json'} 8 data
INFO:nano-graphrag:Writing graph with 8 nodes, 12 edges



📌 Triplette estratte:
"LA FRANCIA" --["La Francia and Il Regno Unito formed an alliance during World War II to combat shared enemies."]--> "IL REGNO UNITO"
"LA FRANCIA" --["La Francia and Gli Stati Uniti were allies during World War II."]--> "GLI STATI UNITI"
"LA FRANCIA" --["La Francia was part of Le forze alleate working together against the Axis powers."]--> "LE FORZE ALLEATE"
"IL REGNO UNITO" --["Il Regno Unito and Gli Stati Uniti were allies, collaborating during World War II."]--> "GLI STATI UNITI"
"IL REGNO UNITO" --["Il Regno Unito was part of Le forze alleate, collaborating with other Allied countries during World War II."]--> "LE FORZE ALLEATE"
"LA SECONDA GUERRA MONDIALE" --["La Germania was a major opposing force to the Allies during World War II."]--> "LA GERMANIA"
"LA SECONDA GUERRA MONDIALE" --["L'Unione Sovietica played a key role in the Allies' victory over Nazi Germany in World War II."]--> "L'UNIONE SOVIETICA"
"GLI STATI UNITI" --["Gli Stati Uniti were members of Le

In [1]:
import mgclient
import networkx as nx

# ✅ Carica grafo da GraphML generato da GraphRAG
G = nx.read_graphml("./nano_cache/graph_chunk_entity_relation.graphml")

# ✅ Estrai le triplette (soggetto, relazione, oggetto)
triples = []
for u, v, data in G.edges(data=True):
    subject = u.replace('"', '')  # rimuove virgolette doppie
    obj = v.replace('"', '')
    relation = data.get("description", "relazione_non_specificata").replace('"', '')
    triples.append((subject, relation, obj))

# ✅ Connessione a Memgraph
conn = mgclient.connect(host="localhost", port=7687)
cursor = conn.cursor()

# ✅ Inserimento delle triplette in Memgraph
for s, r, o in triples:
    cursor.execute("""
        MERGE (a:Entity {name: $s})
        MERGE (b:Entity {name: $o})
        MERGE (a)-[:RELATION {description: $r}]->(b)
    """, {"s": s, "o": o, "r": r})

conn.commit()
conn.close()

print("✅ Triplette inserite in Memgraph.")

ModuleNotFoundError: No module named 'mgclient'

In [ ]:
cursor.execute("MATCH (a)-[r]->(b) RETURN a.name, r.type, b.name")
for row in cursor:
    print(row)

In [ ]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

cursor.execute("""
  MATCH (a)-[r]->(b) 
  RETURN a.name, r.type, b.name
""")

context = "\n".join([f"{a} {r} {b}" for a, r, b in cursor])

response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "Rispondi solo in base al contesto fornito."},
    {"role": "user", "content": f"Contesto:\n{context}\n\nDomanda: Quali alleanze ha stretto la Francia?"}
  ]
)

print("Risposta LLM:")
print(response['choices'][0]['message']['content'])
